In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.결측치 처리

| 버전 | 결측치 처리 방식                       | 함수 이름                  |
|------|----------------------------------------|----------------------------|
| A    | 모든 결측치를 **0**으로 대체            | `filling_zero`             |
| B    | 모든 결측치를 **평균값**으로 대체        | `filling_mean`             |
| C    | 모든 결측치를 **중앙값**으로 대체        | `filling_median`           |
| D    | 모든 결측치를 **-1**로 대체             | `filling_minus1`           |
| E    | 모든 결측치를 **클러스터별 평균**으로 대체 | `filling_cluster_mean`     |
| F    | 모든 결측치를 **KNN 기반 보간**으로 대체  | `filling_knn`              |
| G    | 모든 결측치를 **최빈값**으로 대체        | `filling_mode`             |
| H    | 모든 결측치를 **앞의 값으로 채움**        | `filling_forward`          |
| I    | 모든 결측치를 **뒤의 값으로 채움**        | `filling_backward`         |
| J    | 모든 결측치를 **선형 보간법**으로 대체     | `filling_interpolate`      |
| K    | 모든 결측치를 **무작위 샘플로 대체**       | `filling_random_sample`    |
| L    | 모든 결측치를 **고정값(-999)**으로 대체    | `filling_constant`         |

In [2]:
# 다양한 결측치 처리 전략을 함수로 정의

import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

def filling_zero(df):
    df = df.copy()
    df.fillna(0, inplace=True)
    return df

def filling_minus1(df):
    df = df.copy()
    df.fillna(-1, inplace=True)
    return df

def filling_mean(df):
    df = df.copy()
    for col in df.select_dtypes(include=["number"]).columns:
        df[col] = df[col].fillna(df[col].mean())
    return df

def filling_median(df):
    df = df.copy()
    for col in df.select_dtypes(include=["number"]).columns:
        df[col] = df[col].fillna(df[col].median())
    return df

def filling_cluster_mean(df):
    df = df.copy()
    if 'cluster' not in df.columns:
        raise ValueError("클러스터 정보가 없습니다.")
    for col in df.select_dtypes(include=["number"]).columns:
        if col == 'cluster': continue
        for cl in df['cluster'].unique():
            mean_val = df.loc[df['cluster'] == cl, col].mean()
            df.loc[(df['cluster'] == cl) & (df[col].isnull()), col] = mean_val
    return df

def filling_knn(df, n_neighbors=5):
    df = df.copy()
    numeric_cols = df.select_dtypes(include=["number"]).columns
    imputer = KNNImputer(n_neighbors=n_neighbors)
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])
    return df